In [37]:
import os
import pandas as pd
import optuna
import numpy as np
import openml
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import plotly
import scipy.stats as stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from optuna.samplers import TPESampler, CmaEsSampler
from xgboost import XGBClassifier
from plotly.io import show
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [38]:
dataset_ids = [
    1590,   # Adult
    1510,   # Breast Cancer Wisconsin (Diagnostic)
    1461,   # Bank Marketing
    24,     # Mushroom
    40945   # Titanic
]

In [39]:
def fetch_and_prepare(openml_id):
    global y
    global X
    dataset = openml.datasets.get_dataset(openml_id)
    print(f">>> {dataset.name} (ID: {openml_id})")
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')
    for col in X.select_dtypes(include=['category', 'object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    scaler = MinMaxScaler()
    X_imputed = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)
    if y.dtype == 'category':
        y = LabelEncoder().fit_transform(y)
    zbior = X_imputed['Target'] = y
    return X_imputed

In [40]:
zbior = fetch_and_prepare(40945)

>>> Titanic (ID: 40945)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(zbior.iloc[:, :-1], zbior['Target'], test_size=0.30, random_state=42)

def objective(trial):

    n_neighbors = trial.suggest_int('n_neighbors', 10, 30)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    wyniki = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        X_train_fold = X_train_fold.reset_index(drop=True)
        X_test_fold = X_test_fold.reset_index(drop=True)
        y_train_fold = y_train_fold.reset_index(drop=True)
        y_test_fold = y_test_fold.reset_index(drop=True)

        X_train_fold = list(zip(*[X_train_fold[col] for col in X_train_fold]))
        X_test_fold = list(zip(*[X_test_fold[col] for col in X_test_fold]))

        KNN = KNeighborsClassifier(n_neighbors = n_neighbors)

        model = KNN.fit(X_train_fold, y_train_fold)


        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        auc = roc_auc_score(y_test_fold, y_pred_proba)
                                            
        wyniki.append(auc)

    rezultat = np.mean(wyniki)
    return rezultat

In [42]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
wynik = study.optimize(objective, n_trials=32)

[I 2025-11-14 20:13:12,469] A new study created in memory with name: no-name-6e8a11b4-05e6-4032-8f53-6c001361e8b1
[I 2025-11-14 20:13:12,543] Trial 0 finished with value: 0.9482930589298348 and parameters: {'n_neighbors': 16}. Best is trial 0 with value: 0.9482930589298348.
[I 2025-11-14 20:13:12,619] Trial 1 finished with value: 0.9503442834025204 and parameters: {'n_neighbors': 22}. Best is trial 1 with value: 0.9503442834025204.
[I 2025-11-14 20:13:12,666] Trial 2 finished with value: 0.9491153553091424 and parameters: {'n_neighbors': 17}. Best is trial 1 with value: 0.9503442834025204.
[I 2025-11-14 20:13:12,718] Trial 3 finished with value: 0.9544553339870682 and parameters: {'n_neighbors': 30}. Best is trial 3 with value: 0.9544553339870682.
[I 2025-11-14 20:13:12,765] Trial 4 finished with value: 0.9543821525172167 and parameters: {'n_neighbors': 10}. Best is trial 3 with value: 0.9544553339870682.
[I 2025-11-14 20:13:12,817] Trial 5 finished with value: 0.9503442834025204 and p

In [43]:
study.best_params

{'n_neighbors': 29}

In [44]:
#Defaultowe
KNN = KNeighborsClassifier(n_neighbors = 7)
model = KNN.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.9423209002535927


In [45]:
#Optymalne
KNN = KNeighborsClassifier(n_neighbors = 29)
model = KNN.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.9451606086221471
